In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(ggh4x)
library(patchwork)

# Prepare data for analysis

- Prepared categorical markers

In [ ]:
ii <- readRDS(paste0(SHARE_DIR, "biomarkers_ready.Rds"))
ready <- ii$ready %>% rw() %>% mu(groupedTreatmentType = paste0(unique(strsplit(derived_treatmentType, " ## ")[[1]]), collapse = " ## ")) %>% ug()

- Cohorts 

In [ ]:
cohorts <- 
fread("/mnt/bioinfnas2/immunocomp/shared_reference_data/cohorts/cohorts_ready.csv") %>% 
 se(sampleId, cohort) %>% 
 mu(cohort = ifelse( cohort %in% c("Colon", "Rectum"), "Colorectum", cohort))

- Make treatments based on cohort, treatment name, mechanism, and type

In [ ]:
go_treat <- 
rbind(ready %>% lj(cohorts, by = "sampleId"), ready %>% mu(cohort = "Pan-Cancer")) %>% 
 mu(cohortGo = paste0(cohort, " ## ", derived_treatmentName), group = "treatment")

In [ ]:
go_mechanism <- 
rbind(ready %>% lj(cohorts, by = "sampleId"), ready %>% mu(cohort = "Pan-Cancer")) %>% 
 mu(cohortGo = paste0(cohort, " ## ", derived_treatmentMechanism), group = "mechanism")

In [ ]:
go_type <- 
rbind(ready %>% lj(cohorts, by = "sampleId"), ready %>% mu(cohort = "Pan-Cancer")) %>% 
 mu(cohortGo = paste0(cohort, " ## ", groupedTreatmentType), group = "type" ) %>% 
 fi(groupedTreatmentType %in% c("Chemotherapy", "Immunotherapy", "Targeted therapy", "Hormonal therapy"))

In [ ]:
all <- rbind(ready %>% lj(cohorts %>% se(sampleId, cohort), by = "sampleId")) %>% mu(cohortGo = "Pan-Cancer", group = "type" )

- Remove redundant cohorts and save 

In [ ]:
remove <- c("Pan-Cancer ## Anti-AR", "Pan-Cancer ## Folinic acid ## Platinum ## Pyrimidine (ant)agonist ## Topoisomerase inhibitor", 
           "Pan-Cancer ## Abiraterone", "Pan-Cancer ## Fluorouracil ## Irinotecan ## Leucovorin ## Oxaliplatin", 
           "Unknown primary (e.g. CUP) ## Chemotherapy", 
           "Pancreas PAAD ## Folinic acid ## Platinum ## Pyrimidine (ant)agonist ## Topoisomerase inhibitor", 
           "Prostate ## Hormonal therapy", 
           "Pan-Cancer ## Bevacizumab ## Capecitabine ## Oxaliplatin", 
           "Colorectum ## Anti-VEGF ## Platinum ## Pyrimidine (ant)agonist")

In [ ]:
cohort_maps <- 
c("Pancreas PAAD ## Fluorouracil ## Irinotecan ## Leucovorin ## Oxaliplatin" = "Pancreas PAAD ## FOLFIRINOX",
  "Colorectum ## Bevacizumab ## Capecitabine ## Oxaliplatin" = "Colorectum ## CAPEOX + Bevacizumab")

In [ ]:
go <- 
go_treat %>% 
 bind_rows(go_mechanism) %>% 
 bind_rows(go_type) %>% 
 bind_rows(all) %>% 
 fi(!cohortGo %in% remove) %>% 
 mu(cohortGo = ifelse(cohortGo %in% names(cohort_maps), cohort_maps[cohortGo], cohortGo), 
    pan = grepl("Pan-Cancer", cohortGo))

In [ ]:
fwrite(go, paste0(SHARE_DIR, "fisher_base.csv"))

- Compute and save top mechanisms

In [ ]:
min_patients <- 30

In [ ]:
top_mechanisms <- 
go %>% 
 gb(cohortGo, group) %>% 
 su(ct = n(), no_dcb = sum(non_response), dcb = ct - no_dcb) %>% 
 fi(ct > min_patients, no_dcb >= min_response, dcb >= min_response) %>% 
 ug()

In [ ]:
fwrite(top_mechanisms, paste0(SHARE_DIR, "top_mechanisms.csv"))

# Figures

In [ ]:
top_pan <- top_mechanisms %>% fi(grepl("Pan-Cancer", cohortGo)) 
top_tissue <- top_mechanisms %>% fi(grepl("Pan-Cancer", cohortGo)) 

In [ ]:
p_base <- 
top_mechanisms %>% 
 mu(cohortGoo = ifelse(cohortGo == "Pan-Cancer", "All treatments", cohortGo)) %>% 
 mu(pan = grepl("Pan-Cancer", cohortGo) | cohortGo == "All treatments", 
    cohortGoo = gsub("Pan-Cancer ## ", "", cohortGoo), 
    cohortGoo = gsub("\\(ant\\)agonist", "", cohortGoo), 
    cohortGoo = gsub(" ## ", " / ", cohortGoo)) %>% 
 mu(cohortGooo = ifelse(!pan, sub(" / ", "\n", cohortGoo), cohortGoo))

In [ ]:
#p_base

In [ ]:
fwrite( p_base , paste0(SHARE_DIR, "cohort_names_map.csv"))

In [ ]:
base <- 
p_base %>% 
 ga( response, val, -cohortGo, -group, -ct, -pan, -cohortGoo, -cohortGooo) %>% 
 mu( cohortGo = factor(cohortGo, levels = p_base$cohortGo[order(p_base$ct)]), 
     group = factor(group, levels = c("treatment", "mechanism", "type")), 
     response = factor(response, levels = rev(c("dcb", "no_dcb")))) %>% 
 group_by(cohortGo, cohortGoo) %>%
 arrange(cohortGo, rev(response)) %>%
 mutate(pos = cumsum(val) - 0.5 * val)

- Figure function

In [ ]:
options(repr.plot.width = 12, repr.plot.length = 10)

In [ ]:
colors <- c("no_dcb" = "#e52f28", "dcb" = "#7AABD3")

In [ ]:
extra_theme <- 
theme(legend.position = "none") + 
  theme(axis.title.x = element_blank(),axis.title.y = element_blank()) + 
  theme(axis.text.y = element_text(size = 12), plot.title = element_text(size = 13)) 

In [ ]:
plter <- function(d_base, title = "X", text_size = 4){
 df_totals <- d_base %>% gb(cohortGo) %>% su(total = sum(val)) 

 ggplot(d_base, aes(x = val, y = cohortGo, fill = response)) +
  geom_bar(stat = "identity", color = "black", alpha = .8) +
  labs(title = title) +
  geom_text(aes(label = val, x= pos), color = "black", position = position_dodge(), size = text_size) + 
  scale_fill_manual( values = colors) +  
  geom_text(data = df_totals, aes(y = cohortGo, x = total, label = total), inherit.aes = FALSE, hjust = -.2, size = 4) + 
  scale_x_continuous(expand = expansion(c(0, 0.1))) + 
  go_theme + extra_theme
}

- Go go plots

In [ ]:
p1 <- plter(base %>% fi(!pan, group == "treatment"), "Cohort Specific: Drug Treatment")
p2 <- plter(base %>% fi(!pan, group == "mechanism"), "Cohort Specific: Drug Mechanism") 
p3 <- plter(base %>% fi(!pan, group == "type"), "Cohort Specific: Drug Type") + labs(title = "Cohort Specific: Drug Type", x = "Number of Patients") + go_theme + extra_theme + theme(legend.position = "none",axis.title.y = element_blank()) 
p4 <- plter(base %>% fi(pan, group == "treatment"), "Pan-Cancer: Drug Treatment") 
p5 <- plter(base %>% fi(pan, group == "mechanism"), "Pan-Cancer: Drug Mechanism") 
p6 <- plter(base %>% fi(pan, group == "type"), "Pan-Cancer: Drug Type")  + labs(title = "Pan-Cancer: Drug Type", x = "Number of Patients") + go_theme + extra_theme + theme(legend.position = "none",axis.title.y = element_blank()) + scale_x_continuous(expand = expansion(c(0, 0.2))) 

- Drug Mechanism

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 10)

In [ ]:
cohort <- (p1 / p2 / p3 ) + plot_layout(heights = c(5, 7, 8)) 

In [ ]:
pan_cancer <- (p4 / p5 / p6 ) + plot_layout(heights = c(6, 6, 3)) 

In [ ]:
oo = (cohort | pan_cancer ) + plot_layout(widths = c(1,1)) #+ plot_annotation(title = "Overall Title", subtitle = "Subtitle here")

In [ ]:
oo

In [ ]:
ggsave( "cohorts.png", plot = oo, width = 12, height = 10)

In [ ]:
list.files()